In [1]:
import torch
from torch import Tensor
from torch import nn 
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
from utility import *

torch.manual_seed(7)
np.random.seed(seed=7)

In [2]:
import dlc_bci

train_input_100 , train_target_100 = dlc_bci.load(root = './data_bci_100Hz', download = False)
test_input_100 , test_target_100 = dlc_bci.load(root = './data_bci_100Hz', download = False, train = False)

train_input_1000 , train_target_1000 = dlc_bci.load(root = './data_bci_1000Hz', download = False, one_khz = True)
test_input_1000 , test_target_1000 = dlc_bci.load(root = './data_bci_1000Hz', download = False, train = False, one_khz = True)

print("Train input 100 Hz: {:d}x{:d}x{:d}".format(*(s for s in train_input_100.size())))
print("Train target 100 Hz: {:d}".format(*(s for s in train_target_100.size())))
print("Test input 100 Hz: {:d}x{:d}x{:d}".format(*(s for s in test_input_100.size())))
print("Test target 100 Hz: {:d}".format(*(s for s in test_target_100.size())))
print("")
print("Train input 1000 Hz: {:d}x{:d}x{:d}".format(*(s for s in train_input_1000.size())))
print("Train target 1000 Hz: {:d}".format(*(s for s in train_target_1000.size())))
print("Test input 1000 Hz: {:d}x{:d}x{:d}".format(*(s for s in test_input_1000.size())))
print("Test target 1000 Hz: {:d}".format(*(s for s in test_target_1000.size())))

Train input 100 Hz: 316x28x50
Train target 100 Hz: 316
Test input 100 Hz: 100x28x50
Test target 100 Hz: 100

Train input 1000 Hz: 316x28x500
Train target 1000 Hz: 316
Test input 1000 Hz: 100x28x500
Test target 1000 Hz: 100


# Network definition

This conv2d network is more inspired from a classic conv2d net that is used for image recognition 

In [3]:
from models import *

In [4]:
def compute_nb_errors(model, data_input, data_target, batch_size):
    nb_errors = 0
    Ndata = len(data_input[:, 0, 0, 0])
    model.eval()
    print('data_target', data_target.shape)
    print('data_input', data_input.shape)
    
    for b_start in range(0, Ndata, batch_size):
        bsize_eff = batch_size - max(0, b_start+batch_size-Ndata)  # boundary case
        batch_output = model.forward(data_input.narrow(0, b_start, bsize_eff))  # is Variable if data_input is Variable
        if len(list(batch_output.size()))>1 and batch_output.size(1) > 1:
            # as many ouputs as there are classes => select maximum output
            nb_err_batch = (batch_output.max(1)[1] != data_target.narrow(0, b_start, bsize_eff)).long().sum()
            # overflow problem if conversion to Long Int not performed, treated as short 1-byte int otherwise!!
        else:
            # output is a scalar in [0, 1]
            print('b_size_eff', bsize_eff)
            nb_err_batch = batch_output.round().sub(data_target.narrow(0, b_start, bsize_eff)).sign().abs().sum()
        
        nb_errors += nb_err_batch
    if isinstance(nb_errors, Variable):
        nb_errors = nb_errors.data[0]
    return nb_errors

# Preprocessing and Data Augmentation

In [ ]:
import numpy as np

preprocessed_input_train, preprocessed_input_validation, preprocessed_input_train_target, preprocessed_input_validation_target, kept_channels_idx = preprocessing_train(train_input_1000, train_target_1000, subsampling_frequency='125Hz', window=True,denoize=False, addGaussianNoise=False, reduceChannels=False, cutEnd = False)
preprocessed_input_test = preprocessing_test(test_input_100, kept_channels_idx, subsampling_frequency='125Hz', window=True,denoize = False, reduceChannels=False, cutEnd=False)

#Remove Noise
#preprocessed_input_train = denoisedSignals(preprocessed_input_train)
#preprocessed_input_validation = denoisedSignals(preprocessed_input_validation)
#preprocessed_input_test = denoisedSignals(preprocessed_input_test)
#add random noise
#preprocessed_input_train = whiteNoise(preprocessed_input_train)
#preprocessed_input_validation = whiteNoise(preprocessed_input_validation)
#preprocessed_input_test = whiteNoise(preprocessed_input_test)

print('train', preprocessed_input_train.shape)
print('test', preprocessed_input_test.shape)
print('validation', preprocessed_input_validation.shape)

labels_train = torch.from_numpy(preprocessed_input_train_target)
labels_test = test_target_100
labels_validation = torch.from_numpy(preprocessed_input_validation_target)

preprocessed_input_train = torch.from_numpy(preprocessed_input_train).float()
preprocessed_input_test = torch.from_numpy(preprocessed_input_test).float()
preprocessed_input_validation = torch.from_numpy(preprocessed_input_validation).float()

preprocessed_input_train_target = torch.from_numpy(preprocessed_input_train_target)
preprocessed_input_validation_target = torch.from_numpy(preprocessed_input_validation_target)

Ntrain = len(preprocessed_input_train[:,0,0])
Ntest = len(preprocessed_input_test[:,0,0])
Nvalidation = len(preprocessed_input_validation[:,0,0])

Nchannels = len(preprocessed_input_train[0,:,0])
Nsamples_100 = preprocessed_input_train.size(-1)

print('Ntrain = ', Ntrain)
print('Ntest = ', Ntest)
print('Nvalidation = ', Nvalidation)

train_input = Variable(preprocessed_input_train.view(Ntrain, 1, Nchannels, Nsamples_100))
validation_input = Variable(preprocessed_input_validation.view(Nvalidation, 1, Nchannels, Nsamples_100), requires_grad=False)
test_input = Variable(preprocessed_input_test.contiguous().view(Ntest, 1, Nchannels, Nsamples_100), requires_grad=False)

train (18300, 28, 8)
test (600, 28, 8)
validation (976, 28, 8)
Ntrain =  18300
Ntest =  600
Nvalidation =  976


# Network training and testing
Non-linearity: elu  


|criterion | optimizer | lr  | momentum | batch size | Nepochs | Train acc. | Test acc.|
|----------|-----------|-----|----------|------------|---------|------------|----------|
| BCE  | Adam  |1e-1 | def. | 15 | 150 | 86.4 | 61.4 | 
| BCE  | Adam  |1e-1 | def. | 20 | 150 | 99.8 | 79.5 | 
| BCE  | SGD   | 1e-2 | 0.85 | 20 | 150 | 98.9  | 61.5 | 
| CE   | Adam  | 1e-2 | def. | 20 | 150 | 98.4  |  70.5 | 
| CE   | SGD   | 1e-2 | 0.85 | 20 | 150 | 99.1 | 75.1 |


Non-linearity: ReLU

In [ ]:
import torch.optim as optim
import math

# Train network 
criterion = nn.BCELoss()
#criterion = nn.CrossEntropyLoss()
#criterion = nn.PoissonNLLLoss()
#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.SmoothL1Loss() #interesting ... but does not converge
#criterion = nn.MSELoss() #0.83 but unstable

if isinstance(criterion, nn.CrossEntropyLoss):
    train_target = Variable(preprocessed_input_train_target)  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, requires_grad=False) # convert to float
    test_target = Variable(test_target_100, requires_grad=False)
    Noutputs = 2
    
elif isinstance(criterion, nn.NLLLoss):
    train_target = Variable(preprocessed_input_train_target)  # keep long tensors
    validation_target = Variable(preprocessed_input_validation_target, requires_grad=False) # convert to float
    test_target = Variable(test_target_100, requires_grad=False )
    Noutputs = 2
    
else:
    train_target = Variable(preprocessed_input_train_target.float()) # convert to float
    validation_target = Variable(preprocessed_input_validation_target.float(), requires_grad=False) # convert to float
    test_target = Variable(test_target_100.float(), requires_grad=False)
    Noutputs = 1
        
model = conv2DNet_1(Nchannels, Nsamples_100, Noutputs) #from classic knowledge of image segmentation  
#model = conv2DNet_2(Nchannels, Nsamples_100, Noutputs) #from litterature 
#model = conv2DNet_3(Nchannels, Nsamples_100, Noutputs) #from PdM

#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.50)
optimizer = optim.Adam(model.parameters())
#optimizer = optim.Adagrad(model.parameters())
#optimizer = optim.Adamax(model.parameters())
#optimizer = optim.ASGD(model.parameters())
#optimizer = optim.RMSprop(model.parameters())
#optimizer = optim.Rprop(model.parameters())

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, verbose=True)

batch_size = 15
Nbatches = int(math.ceil(Ntrain/batch_size))
Nepochs = 30
Nrep = 1

train_errors = torch.Tensor(Nepochs).zero_()
test_errors = torch.Tensor(Nepochs).zero_()
validation_errors = torch.Tensor(Nepochs).zero_()

ep_loss = torch.Tensor(Nepochs).zero_()

for i_rep in range(Nrep):
    for i_ep in range(Nepochs):
        for b_start in range(0, Ntrain, batch_size):
            bsize_eff = batch_size - max(0, b_start+batch_size-Ntrain)  # boundary case
            model.train()
            model.zero_grad()
            output = model(train_input.narrow(0, b_start, bsize_eff))
            batch_loss = criterion(output, train_target.narrow(0, b_start, bsize_eff))            
            ep_loss[i_ep] += batch_loss.data[0]
            batch_loss.backward()
            optimizer.step()
        
        scheduler.step(ep_loss[i_ep])
        
        nb_train_errs = compute_nb_errors(model, train_input, train_target, batch_size)
        nb_validation_errs = compute_nb_errors(model, validation_input, validation_target, batch_size)
        nb_test_errs = compute_nb_errors(model, test_input, test_target, batch_size)
        
        print("Epoch Number : ", i_ep)
        print("\t Training accuracy: ", (100*(Ntrain-nb_train_errs)/Ntrain))
        print("\t Validation accuracy ",(100*(Nvalidation-nb_validation_errs)/Nvalidation)) 
        print("\t Test accuracy ",(100*(Ntest-nb_test_errs)/Ntest))
        
        print("\t Epoch Loss ", ep_loss[i_ep])
        
        train_errors[i_ep] = nb_train_errs
        test_errors[i_ep] = nb_test_errs
        validation_errors[i_ep] = nb_validation_errs

torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])


C:\Users\DavidCleres\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15

torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15

torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15, 64, 32, 12])
torch.Size([15

...

torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size(

b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 

C:\Users\DavidCleres\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



data_target torch.Size([976])
data_input torch.Size([976, 1, 28, 8])
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12])
b_size_eff 15
torch.Size([15, 64, 32, 12]

RuntimeError: length out of range

In [ ]:
plt.plot(np.array(train_errors))
plt.plot(np.array(validation_errors))
plt.plot(np.array(test_errors))
plt.show()

In [ ]:
plt.plot(100*(Ntrain-np.array(train_errors))/Ntrain)
plt.plot(100*(Nvalidation-np.array(validation_errors))/Nvalidation)
plt.plot(100*(Ntest-np.array(test_errors))/Ntest)

plt.show()